In [23]:
from dotenv import load_dotenv
import os
from pyairtable import Api
from pyairtable.formulas import match
from datetime import datetime
from models import *
import pandas as pd
from pathlib import Path


load_dotenv()
KEY = os.getenv('KEY')
app = "app7KsgYl2jhOnYg7"

api = Api(KEY)
base = api.base(app)

home_dir = Path("../")
data_dir = home_dir / "data"
raw_dir = home_dir / "raw"


## Formulae

## Get Reaction Names

In [ ]:
reaction_list = list(raw_dir.rglob("*.xlsx"))
reactions = [file.name.replace(".xlsx", "") for file in reaction_list]

## Update Reaction Table

In [ ]:
for rxn in reactions:

    rxn_split = rxn.split("_")

    assay = rxn_split[rxn.count("_")]
    
    date = rxn_split[0]
    date = "-".join([date[:4], date[4:6], date[6:8]])
    date = datetime.strptime(date, "%Y-%m-%d")

    reader = rxn_split[1]

    tech_initials = rxn_split[2]

    tech_formula = match({"initials": tech_initials})
    technician = Technician.first(formula=tech_formula)

    rxn_formula = match({"rxn_name": rxn})

    if not Reaction.first(formula=rxn_formula):
        reaction = Reaction(
            rxn_name = rxn,
            assay = assay,
            date = date,
            technician = technician,
            reader = reader,
            temperature = 42
        )

        reaction.save()
    else:
        print(f"Entry already exists for rxn: {rxn}")

## Load in raw data

In [ ]:
# raw_files = list(data_dir.rglob("raw.parquet"))

# df_list = []
# for file in raw_files:
#     df = pd.read_parquet(file)
#     df_list.append(df)
# df = pd.concat(df_list)

## Update Raw Table

In [ ]:
# rxn_string = ""

# for _, row in df.iterrows():
#     sample_id = row.get("Sample IDs")
#     if pd.isna(sample_id):
#         continue

#     sample_formula = match({"sample_id": sample_id})
#     sample = Sample.first(formula=sample_formula)

#     rxn = row.get("Reaction")
#     if rxn != rxn_string:
#         reaction = Reaction.first(formula=rxn_formula)
#         rxn_string = rxn

#     dilution = row.get("Dilutions")
#     well = row.get("Wells")
#     time = row.get("Time")
#     value = row.get("RFU")

#     raw_formula = match({
#         # "sample": sample,
#         "reaction": reaction,
#         # "dilution": dilution,
#         "well": well,
#         "time": time,
#     })

#     if not Raw.first(formula=raw_formula):
#         raw = Raw(
#             sample = sample,
#             reaction = reaction,
#             dilution = dilution,
#             well = well,
#             time = time,
#             value = value
#         )
#         raw.save()

## Load in Results data

In [33]:
result_files = list(data_dir.rglob("calcs.parquet"))

df_list = []
for file in result_files:
    df = pd.read_parquet(file)
    df_list.append(df)
df = pd.concat(df_list)

## Update Results

In [ ]:
rxn_string = ""
samples = [sample.sample_id for sample in Sample.all()]
# print(samples)

df = df[df['Sample IDs'].isin(samples)]

for _, row in df.iterrows():
    sample_id = row.get("Sample IDs")

    if pd.isna(sample_id): continue

    sample_formula = match({"sample_id": sample_id})
    sample = [Sample.first(formula=sample_formula)]

    if not sample: 
        print(f"No Airtable entry for sample {sample_id}.")
        continue

    rxn = row.get("Reaction")
    if rxn != rxn_string:
        reaction = [Reaction.first(formula=rxn_formula)]
        rxn_string = rxn

    dilution = row.get("Dilutions")
    well = row.get("Wells")
    mpr = row.get("MPR")
    ms = row.get("MS")
    ttt = row.get("TtT")
    raf = row.get("RAF")
    auc = row.get("AUC")

    result_formula = match({
        "reaction": reaction,
        "well": well,
    })

    print(Result.first(formula=result_formula))

    if not Result.first(formula=result_formula):
        result = Result(
            sample = sample,
            reaction = reaction,
            dilution = dilution,
            well = well,
            mpr = mpr,
            ms = ms,
            ttt = ttt,
            raf = raf,
            auc = auc
        )
        result.save()
        print("Entry saved.")
    else:
        print("An entry already exists for this.")

TypeError: LinkField value must be <class 'list'>; got <class 'models.Sample'>